# Lab 1- Information theory

This lab has a few goals designed to get you comfortable with working with python and playing with the basiscs of information theory.

Sections:
1. Entropy
2. Mutual information

## Background

We are going to assume a basic familiarity with python. If you need an introduction, see the introduction notebook at the beginning of this book. That should get you enough familiarity to get started.

For this lab we won't be working with our main library, _explorationlib_. We'll use a few standard libraries to both get more comfortable with python and to refresh the concepts of information theory discussed in class.

### **What is entropy again?**

- According to Shannon's definition, the entropy $H(X)$ of a discrete random variable is: $H(X)= \sum_{x \in X} p(x) \log _2 p(x)^{-1}$.
- Here we are assuming that the discrete variable is binary, hence the use of $\log _2$
- A key aspect of the definition entropy is the 'surprise' termn, $p(x)^{-1}$. The more surprising an outcome is, the more valuable that bit of information is. We will return to this later in class.

### **Entropy between variables**.

- The _joint entropy_ between two discrete random variables, $X$ and $Y$, is just an expansion of the regular concept of entropy: $H(X, Y)= \sum_{x \in X} p(x,y) \log _2 p(x,y)^{-1}$.
- Note that in here, $p(x,y)$ is the joint probability distribution between the variables.
- Expanding out from the rules of conditional probabilities (thank you Mr. Bayes), the joint entropy can be expanded as the product of the entropy of the primary variable and the conditional entropy of the two variables: $H(X,Y) = H(X) + H(Y|X)$. _Pay attention to the ordering of terms, it gets important later_.

### **Mutual information between variables**.

- Sticking with the idea of joint entropy, the _conditional entropy_ $H(X|Y)$ reflects the residual entropy of $X$ after you have knowledge about $Y$. This is expressed as: $H(X|Y)=H(X)-I(X;Y)$.
- We call this second term, $I(X;Y)$ the _mutual information_ between $X$ and $Y$. It is the information provided by Y about X. If you were in statistics and working with continuous variables, this would be the correlation (or covariance).
- We can rewrite the equation above as $I(X;Y) = H(X) - H(X|Y) = \sum_{x \in X, y \in Y} p(x,y) \log _2 \frac{p(x,y)}{p(x)p(y)}$

### **Transfer entropy between variables**.

- In statistics (including information theory), causality follows Humeian logic: cause temporally preceeds effect. Thus, the term 'causality' in a statistical sense really just looks at the asymmetry in cross-correlations between two or more time series.
- Information theory relies on _transfer entropy_ $T(X \rightarrow Y)$ to estimate this statistical relationship.
- Trasfer entropy captures how much information the source has transfered to the receiver: $T(X \rightarrow Y) = I(Y_{f} ; X_p | Y_p) = H(Y_f|Y_p) - H(Y_f|X_p, Y_p)$, which can be rewritten as $T(X \rightarrow Y) =  \sum_{y_f \in Y_f, x_p \in X_p, y_p \in Y_p} p(y_f,x_p,y_p) \log _2 \frac{p(y_f,x_p|y_p)}{p(y_f|y_p)p(x_p|y_p)}$

## Section - Setup

Use the rocket-shaped button to open this assignment in a colab. Once it is open, if it is open, run all the cells. Read each cell, then run it, that is. It's that simple.

We will discuss a different way to work with notebooks for the Exercises, but for now just load in Colab.

For this lab we'll load some standard libraries for working with mutual information.

In [ ]:
# Import the general libraries we will be using
import numpy as np
import matplotlib.pyplot as plt

## Section 1 - Entropy

- For this section we will simulate two cells, a presynaptic and postsynaptic cell. The data will consist of spikes. We will vary the degree of dependency between the neurons and look at their relative entropies.
- One way to achieve this is to use a Poisson process to simulate the neurons. Here, the firing of each neuron at each time step is modeled as a binary event that occurs with some probability. The influence of the first neuron on the second is represented by increasing the firing probability of the second neuron when the first one fires.

In [ ]:
# Time and rates
total_time_sec = 5
sampling_rate_hz = 1000
n_samples = total_time_sec * sampling_rate_hz

# Probabilities
base_firing_prob = 0.02
influence = 0.03
noise_level = 0.01

def simulate_neuron_spikes(n_samples, base_firing_prob, influence, noise_level, neuron1=None):
    """
    Simulate the spiking of a neuron.

    If neuron1 is not None, then this neuron is influenced by neuron1.
    """
    spikes = np.zeros(n_samples)

    for i in range(n_samples):
        firing_prob = base_firing_prob

        if neuron1 is not None and neuron1[i] == 1:
            firing_prob += influence

        firing_prob += noise_level * np.random.randn()

        if np.random.rand() < firing_prob:
            spikes[i] = 1

    return spikes

# Simulate the two neurons
neuron1 = simulate_neuron_spikes(n_samples, base_firing_prob, 0, noise_level)
neuron2 = simulate_neuron_spikes(n_samples, base_firing_prob, influence, noise_level, neuron1)

print("Neuron 1 spike train:", neuron1)
print("Neuron 2 spike train:", neuron2)


What we have created are two binary vectors, _neuron1_ and _neuron2_, that represent the spike trains of two neurons over 5 seconds at a 1 kHz sampling rate.

Each time step is a binary event where 1 represents a spike and 0 represents no spike. The base firing probability is 2%, but this is modified by two factors: the influence of the first neuron on the second, and some independent noise.

The _influence_ parameter controls how much the first neuron affects the second: whenever the first neuron fires, the firing probability of the second neuron is increased by the influence factor. The _noise level_ parameter controls the amount of independent noise in the firing probabilities. This noise is modeled as Gaussian noise and is independent for each neuron and each time step.

### Visualizing our two neurons

We can use a raster plot to visualize the spike times of our neurons. For this we will use matplotlib.

In a raster plot, each row corresponds to a different repetition of the experiment (a different neuron in this case), and the x-axis represents time. Each small vertical line (marker) represents a spike.

In [ ]:
def plot_spikes(neuron1, neuron2, sampling_rate_hz):
    """
    Plot the spikes of the two neurons as a raster plot.
    """
    time_points = np.arange(len(neuron1)) / sampling_rate_hz

    fig, ax = plt.subplots(2, 1, figsize=(10, 5), sharex=True, sharey=True)

    # Neuron 1
    ax[0].eventplot(time_points[neuron1 == 1], color='black')
    ax[0].set_title("Neuron 1")
    ax[0].set_ylabel("Spikes")

    # Neuron 2
    ax[1].eventplot(time_points[neuron2 == 1], color='black')
    ax[1].set_title("Neuron 2")
    ax[1].set_ylabel("Spikes")
    ax[1].set_xlabel("Time (sec)")

    plt.tight_layout()
    plt.show()

# Run the plotting function
plot_spikes(neuron1, neuron2, sampling_rate_hz)

Let's now calculate the entropy of each neuron. For this we will use a simple function that follows the equations for entropy in the Background and the reading.

In [ ]:
def calculate_entropy(neuron):
    """
    Calculate the entropy of the spike train of a neuron.
    """
    # Calculate the probabilities of 0 and 1
    p_0 = np.sum(neuron == 0) / len(neuron)
    p_1 = np.sum(neuron == 1) / len(neuron)

    # Use the formula for entropy of a binary variable
    # Note: When p_0 or p_1 is 0, their contribution to the entropy is 0
    entropy = 0
    if p_0 > 0:
        entropy -= p_0 * np.log2(p_0)
    if p_1 > 0:
        entropy -= p_1 * np.log2(p_1)

    return entropy


Now let's see how our neurons are doing.

In [ ]:
# Calculate and print the entropies
entropy_neuron1 = calculate_entropy(neuron1)
entropy_neuron2 = calculate_entropy(neuron2)

print(f"Entropy of Neuron 1: {entropy_neuron1}")
print(f"Entropy of Neuron 2: {entropy_neuron2}")


### Question 1.1

Increase the baseline firing rate parameter, *base_firing_prob*, by a factor of 10 to 0.2. What happens to the entropies of the neurons? Explain why this effect may or may not occur.

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Question 1.2

Put the baseline firing rate parameter back to 0.02. Increase the influence parameter, from neuron 1 to neuron 2, by a factor of 10 to 0.3. What happens to the entropies of the neurons? Explain why this effect may or may not occur.

In [ ]:
# Write your answer here as a comment. Explain yourself.

## Section 2 - Mutual information

We have our two happy little neurons firing. We've played with the parameters to see how it impacts their relative entropy. Now let's go back to the original parameter settings and see about estimating the mutual information between our neurons.

Recall that to estimate the mutual information, we need to estimate the conditional entropy for $X$ on $Y$. For binary variables things are a bit easier because $H(X|Y) = H(Y) - H(X,Y)$. Which means we only need to get both independnet entropies and the joint entropy. This leaves us with this simple and beautiful equation: $I(X; Y) = H(X) + H(Y) - H(X, Y)$.

We can easily write a function to do this.

In [ ]:
def calculate_joint_entropy(neuron1, neuron2):
    """
    Calculate the joint entropy of the spike trains of two neurons.
    """
    # Calculate the probabilities of each combination of 0 and 1
    p_00 = np.sum((neuron1 == 0) & (neuron2 == 0)) / len(neuron1)
    p_01 = np.sum((neuron1 == 0) & (neuron2 == 1)) / len(neuron1)
    p_10 = np.sum((neuron1 == 1) & (neuron2 == 0)) / len(neuron1)
    p_11 = np.sum((neuron1 == 1) & (neuron2 == 1)) / len(neuron1)

    # Use the formula for entropy of a binary variable
    # Note: When p_ij is 0, its contribution to the entropy is 0
    joint_entropy = 0
    if p_00 > 0:
        joint_entropy -= p_00 * np.log2(p_00)
    if p_01 > 0:
        joint_entropy -= p_01 * np.log2(p_01)
    if p_10 > 0:
        joint_entropy -= p_10 * np.log2(p_10)
    if p_11 > 0:
        joint_entropy -= p_11 * np.log2(p_11)

    return joint_entropy

def calculate_mutual_information(neuron1, neuron2):
    """
    Calculate the mutual information between the spike trains of two neurons.
    """
    entropy_neuron1 = calculate_entropy(neuron1)
    entropy_neuron2 = calculate_entropy(neuron2)
    joint_entropy = calculate_joint_entropy(neuron1, neuron2)

    # Use the formula for mutual information
    mutual_information = entropy_neuron1 + entropy_neuron2 - joint_entropy

    return mutual_information




So let's see what the mutal information is when we re-simulate our neurons back at the original parameter settings.

In [ ]:
# Time and rates
total_time_sec = 5
sampling_rate_hz = 1000
n_samples = total_time_sec * sampling_rate_hz

# Probabilities
base_firing_prob = 0.02
influence = 0.03
noise_level = 0.01

# Simulate the two neurons
neuron1 = simulate_neuron_spikes(n_samples, base_firing_prob, 0, noise_level)
neuron2 = simulate_neuron_spikes(n_samples, base_firing_prob, influence, noise_level, neuron1)


# Calculate and print the mutual information
mutual_information = calculate_mutual_information(neuron2, neuron1)
print(f"Mutual Information: {mutual_information}")

This reflects the mutual dependence between the two neurons. You can probably guess what questions we will ask about this.

### Question 2.1

Increase the baseline firing rate parameter, *base_firing_prob*, by a factor of 10 to 0.2. What happens to the mutual information of the neurons? Explain why this effect may or may not occur.

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Question 2.2

Put the baseline firing rate parameter back to 0.02. Increase the influence parameter, from neuron 1 to neuron 2, by a factor of 10 to 0.3. What happens to the mutual information of the neurons? Explain why this effect may or may not occur.

In [ ]:
# Write your answer here as a comment. Explain yourself.

### Question 2.3

Take the last set of simulations that you ran (with the influence set at 0.3) and switch the order of the neurons in the calculation of mutual information. What does or does not happen to the mutual information estimate? Why?

In [ ]:
# Write your answer here as a comment. Explain yourself.